# Lecture02

이번시간에는 **Word2Vec**를 설명하면서 **CBOW**과 **skip-gram**도 함께 설명하는 시간을 가져보도록 하겠습니다. Lecture01 시간에 우리는 BoW(bag-of-words)에 배웠었습니다. BoW모델이 가지는 한계점은 복습겸 다시 살펴보면 <Br>
* 단어들의 문맥을 고려하지 않는다.
* 단어들의 본질적인 의미를 고려하지 않는다.
* $V$가 커질수록 연산이 비효율적이다.

정도가 있었죠. 하지만 Word2Vec은 단어들을 벡터로 표현하게 됨으로써 단어자체의 의미를 부여할 수 있게 되었습니다. 그렇다면 어떻게 이것이 가능해졌는지 또 어떠한 방식으로 학습을 시켰는지에 대해서 알아보겠습니다. 
본론에 들어가기에 앞서 이번 설명은 [Word2Vec 논문](https://arxiv.org/pdf/1301.3781.pdf)과 [Beomsu Kim님의 블로그](https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/)의 내용을 참고했습니다.

Word2Vec의 모델은 크게 2가지의 방식으로 이루어져 있습니다. 한가지는 **CBOW(contineous bag of words)** 또 다른 한가지는 **skip-gram**으로 이루어져 있습니다. 

### CBOW
<img src="./img/CBOW.png" width="200" height="200">
CBOW는 주변단어들을 이용해 중심단어가 나타날 확률을 구하는 방식입니다. 문장으로 예를들어 **"나는 일주일에 한번 영화를 본다."** 라는 문장이 있고 단어별로 Token화 하여 **"나는", "일주일에", "한번", "영화를", "본다", "."**으로 이루어 진다고 가정을 해보겠습니다. 여기서 window size를 정해 주어야 하는데 window size란 주변에 단어를 얼마나 볼 것인지를 정해주는 것입니다. 만약 window size를 2라고 한다면 중심단어 양쪽으로 2개의 주변단어를 선택한다는 뜻입니다.

따라서 window size가 2이며 중심단어가 **"한번"**이라한다면 **"나는", "일주일에", "영화를", "본다"**의 주변단어를 이용하여 중심단어가 나올 확률을 계산해 주게 되는 것입니다. 이 예시로 본다면 이게 가능하다고??라는 생각이 들수도 있지만 Word2Vec의 기본적인 idea는 문맥에서 어떠한 단어가 출현할 가능성이 유사한 단어끼리는 비슷하다는 전제가 있습니다. 그렇기 때문에 문맥 **"나는 일주일에 한번 영화를 본다"**에서 **"한번"**과 혹은 **"두번"** 또는 **"여러번"**이 나올 확률적 결과가 비슷하다는거죠. 

그럼 지금부터 바로 Tensorflow를 이용한 CBOW-Word2Vec 모델을 구현해 보겠습니다. 오늘 사용할 데이터셋은 20News를 사용하겠습니다. 20News dataset은 기본적으로 **Scikit-learn** 패키지에서 이용할 수 있습니다. 

### Load data
20News 데이터는 20가지 종류의 뉴스데이터입니다. class들은 종교, 하키, 야구, $\cdots$ , 하드웨어 등이 있으며 이메일 형식으로 이루어져 있습니다.

In [98]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import tensorflow as tf
import datetime
import re
newsgroups_train = fetch_20newsgroups(subset='train')

In [14]:
raw_text = newsgroups_train["data"]

### preprocessing of data
Word2Vec에서도 전처리를 해야할까요?? Word2Vec은 단어들의 표현을 나타내기 위한 모델입니다. 만약에 이전시간과 같이 전처리를 한다면 stop-words의 단어들은 학습하지 못하게 되겠죠? 또한 이 Word2Vec은 학습한 단어벡터를 이용해서 다른 모델에 적용되어지는 경우가 많습니다. 그렇기 때문에 잘 학습이 된다는 가정하에 단어를 표현할 수 있는 개수가 많아지는 것이 효율적입니다. 따라서 이번 시간에는 **Stemming**과 **stop-words**는 사용하지 않습니다.

In [15]:
def clean_str(s):
    # only include alphanumerics
    s = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", s)
    
    # insert spaces in words with apostrophes
    s = re.sub(r"\'s", " \'s", s)
    s = re.sub(r"\'ve", " \'ve", s)
    s = re.sub(r"n\'t", " n\'t", s)
    s = re.sub(r"\'re", " \'re", s)
    s = re.sub(r"\'d", " \'d", s)
    s = re.sub(r"\'ll", " \'ll", s)
    
    # insert spaces in special characters
    s = re.sub(r",", " , ", s)
    s = re.sub(r"!", " ! ", s)
    s = re.sub(r"\(", " \( ", s)
    s = re.sub(r"\)", " \) ", s)
    s = re.sub(r"\?", " \? ", s)
    
    # only include alphanumerics again
    s = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", s)
    
    # reduce multiple spaces to single spaces
    s = re.sub(r"\s{2,}", " ", s)
    return s.strip().lower()

In [19]:
clean_data = [clean_str(x) for x in raw_text]

### Build vocaburary of data
이제 데이터셋에 대한 단어사전을 만듭니다. 학습속도를 높이기 위해서 오늘은 데이터셋에서 출현빈도 순으로 30000개의 단어들로만 단어사전을 구성하겠습니다.

In [39]:
vocab_size = 30000
word_list = "".join(clean_data).split()
count = [['UNK', -1]]
count.extend(collections.Counter(words).most_common(vocab_size - 1))

In [48]:
dictionary = dict()
for word, _ in count:
    dictionary[word] = len(dictionary)  
print("vocab size :", len(dictionary))

vocab size : 30000


### Words scoring methods
이제 preprocessing을 거친 document에서 word의 표현을 **text** 에서 **id**로 매핑시킵니다. 이 과정에서 단어사전 $V$에 포함되지 않는 단어는 "UNK"의 index number인 0으로 매핑됩니다. 이 시간에 window size는 5를 사용합니다. <br>
만약 "나는 일주일에 한번 영화를 본다"에서 중심단어 즉, Target word가 "나는"일 경우에 왼쪽의 주변단어를 어떻게 처리할 수 있을까요? 각 문서의 왼, 오른쪽에 Padding 을 window size만큼 준다면 해결할 수 있겠죠?

In [53]:
window_size = 5
sent_data = []
for sentence in clean_data:
    data = [0]*window_size
    for word in sentence.split():
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0 
        data.append(index)
    data.extend([0]*window_size)    
    sent_data.append(data)

CBOW 에서 input값은 target word에 대한 주변단어, ouput은 target word입니다. 따라서 우리는 이 형식에 맞게 데이터를 구조화해야겠죠. 우선 주변단어( context) 와 중심단어(target)이 전체 데이터셋에서 몇개의 쌍으로 이루어 질 수 있는지 확인해보겠습니다.

In [57]:
instances = 0
for sentence in sent_data:
    instances += len(sentence)-2*window_size
print("The number of pair context and target words :", instances)

The number of pair context and target words : 4079925


context 와 target의 쌍이 4079925개가 있습니다. 그럼 이제 context와 target 즉 input과 output값으로 나눠주겠습니다.
* np.delete(arr, index, axis) : 배열에서 axis 축을 기준으로 index에 해당하는 value를 삭제

In [68]:
context = np.zeros((instances, window_size * 2 + 1), dtype=np.int32)
target = np.zeros((instances, 1), dtype=np.int32)

k = 0
for sentence in sent_data:
    for i in range(window_size, len(sentence) - window_size):
        context[k] = sentence[i - window_size:i + window_size + 1] 
        target[k] = sentence[i]                
        k += 1
context = np.delete(context, window_size, 1)
print("context :", context[0])
print("target :", target[0])

context : [   0    0    0    0    0    0 4298 1348   19    5]
target : [38]


위의 결과에서 볼 수 있듯이 id가 38인 target word과 window size가 5인 context로 구성이 되어집니다.

### Build model architacture
이제 모델을 구현할 차례입니다. Word2Vec은 단어가 벡터로 표현되어지는 모델입니다. 그렇기 때문에 현재 id값으로 매핑이 되어있는 이 단어들을 표현해줄 수 있는 weight가 필요합니다. 우선적으로 placeholder를 지정해줍니다.

In [120]:
context_window = 2*window_size
batch_size = 64

input_x = tf.placeholder(tf.int32, shape=[batch_size * context_window])
input_y = tf.placeholder(tf.int32, shape=[batch_size, 1])

이제 단어를 표현할 벡터를 지정해 주어야 합니다. 이 벡터들이 학습이 되어지면서 최종적인 단어벡터로 표현될 수 있습니다.
* tf.random_uniform : (a,b)사이의 연속균등분포
* embedding_size : $D$ 차원의 단어벡터

In [121]:
embedding_size = 100

word_embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0))

이제 단어벡터의 초기화까지 마쳤습니다. input_x 는 일련의 단어들의 id값인데 이것은 단어벡터와 연결시켜 주어야겠죠? Tensorflow에서 제공하는 함수 lookup table을 사용하면 쉽게 매핑시킬 수 있습니다. lookup-table에 대해서 좀 더 예시를 들어보면 <br>

**"나는", "일주일에", "한번", "영화를", "본다"**가 모든 데이터이며 embedding size을 100이라 가정한 뒤 문서의 표현이 [0, 1, 2, 3, 4]로 매핑되었다고 해봅시다. 위의 변수 word_embedding 은 5개의 단어에 대한 100차원의 벡터들어 이루어져있겠죠. <br>
lookup table을 이용하게 되면 **"영화를", "본다"** 라는 데이터는 word_embedding의 3, 4번째 index에 해당하는 100차원의 값을 가지고 오게됩니다. 

In [122]:
embed_context = tf.nn.embedding_lookup(word_embeddings, input_x)

위의 모델구조에서 보면 context의 단어들이 sum으로 표현 되어집니다. 하지만 embed_context는 $[batch \, size \times context \, window, embedding \, size]$ 로 이루어져 있기 때문에 context끼리 더하기 위해선 몇가지 일을 더 해주어야 합니다. 
Tensorflow의 **segment_sum** 함수를 이용하여 해결해 보도록 하겠습니다. 우선적으로 context를 구분할 수 있는 배열을 만들어 줍니다.

In [124]:
sum_ids = np.repeat(np.arange(batch_size), context_window)
segment_ids = tf.constant(sum_ids, dtype=tf.int32)

<img src="https://www.tensorflow.org/images/SegmentSum.png" width="50%" height="50%">
위의 이미지는 **"segment_sum"** 함수의 계산방식입니다. 우리의 경우에는 segment_id 값이 context 단어의 개수만큼 나누어져 있겠죠. 이제 context 단어들을 sum하겠습니다.

In [125]:
embed_words = tf.segment_sum(embed_context, segment_ids)

Word2Vec 의 Loss function은 negative sampling을 사용합니다. softmax를 사용하여 확률적 계산을 한다면 $N \times V$만큼의 계산이 들어 비효율적이기 때문에 $k$개 만큼의 sample들을 이용한 **NCE loss (noise-contrastive estimation)**를 사용합니다. 
Word2Vec의 학습방법은 context단어들을 이용해 target word를 찾는것입니다. NCE loss를 사용할 경우 적절한 $k$개의 negative 샘플들을 이용하여 context에서 올바른 target을 예측하는지 안하는지에 대한 **이진분류 목적함수**를 사용합니다.

NCE loss 는 로지스틱 회귀 모델에 관하여 정의 되어집니다. 따라서, 단어사전($V$) 의 각 단어에 대한 가중치(weights)와 편향(biases) 을 정의합니다.<br>
Tensorflow에서는 nce_loss 함수를 사용하여 구현합니다.

In [127]:
softmax_weights = tf.Variable(tf.truncated_normal([vocab_size, embedding_size],
                                                   stddev=1.0 / np.sqrt(embedding_size)))
softmax_biases = tf.Variable(tf.zeros([vocab_size]))

In [128]:
loss = tf.reduce_mean(tf.nn.nce_loss(weights=softmax_weights, biases=softmax_biases,
                                     labels=input_y, inputs=embed_words,
                                     num_sampled=5,
                                     num_classes=vocab_size))

In [135]:
optimizer = tf.train.AdagradOptimizer(0.5).minimize(loss)

이제 학습을 시작합니다. 최종적인 단어의 벡터는 **final_word_embeddings** 변수에 나타냅니다.

In [133]:
total_epoch = 1
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(instances/batch_size)
    start_time = datetime.datetime.now()
    for epoch in range(total_epoch):
        avg_loss = index = 0
        for step in range(total_batch):
            if (index + 1) * batch_size < instances:
                loss_val, _ = sess.run([loss, optimizer], feed_dict=
                                       {input_x:np.reshape(context[index*batch_size:(index+1)*batch_size],(-1)),
                                        input_y:target[index*batch_size:(index+1)*batch_size]})       
                index += 1
            else:
                loss_val, _ = sess.run([loss, optimizer], feed_dict=
                                       {input_x:np.reshape(context[instances-batch_size:], (-1)),
                                        input_y:target[instances-batch_size:]})
                index = 0
                
            avg_loss += loss_val / total_batch
        print("{} epoch ----> loss : {}".format(epoch, avg_loss)) 
    final_word_embeddings = word_embeddings.eval()    
    print("Training time : {}".format(datetime.datetime.now()-start_time))

0 epoch ----> loss : 8.22927537598899
Training time : 0:01:04.291378


### Finish 
이번시간에는 Word2Vec의 CBOW방식을 사용해 보았습니다. 다음시간에는 산출된 벡터들이 어떻게 이용되는지 알아보며 skip-gram에 대해서 알아보도록 하겠습니다.